In [2]:
from LambertSolarSystem import LambertSolarSystem
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
from PyKEP import AU, DAY2SEC, DAY2YEAR, MU_SUN, epoch
from PyKEP.orbit_plots import plot_planet, plot_lambert
import PyKEP as pkp
import numpy as np


system = LambertSolarSystem(87464)

posFunc = system.getPositionsFunction()
velFunc = system.getVelocityFunction()

 # finds the optimal transfer between the planets
 Takes a start value t and two planets


In [4]:
#first_boost = 9594

#vel_at_closest = 0

SEC2YEAR = 1./DAY2YEAR*DAY2SEC
t = 17.32
A = 4; B = 5
planets = np.array((A,B))


planetsPos0 = posFunc(t)[:]
planetsVel0 = velFunc(t)[:]
# Add 3. dimension
pos0 = np.r_[planetsPos0, [np.zeros(7)]]
vel0 = np.r_[planetsVel0, [np.zeros(7)]]
rA = pos0[:,A] * AU
vA = vel0[:,A] * AU / SEC2YEAR
rB = pos0[:,B] * AU
vB = vel0[:,B] * AU / SEC2YEAR

    
mu_star = system.starMass * MU_SUN
mu_planet = system.mass[planets]  * MU_SUN
radius = system.radius[planets] * 1000
safe_radius = radius *1.2

planetA = pkp.planet.keplerian(epoch(0), rA, vA, mu_star, mu_planet[0], radius[0], safe_radius[0], 'four')
planetB = pkp.planet.keplerian(epoch(0), rB, vB, mu_star, mu_planet[1], radius[1], safe_radius[1], 'five')

time_array = np.linspace(4.1,8,10000)
v_list = []
# ----------------------Find best lambert sol ----------------------

for time_use in time_array:
    t1 = epoch(0)            #relative to t0
    t2 = epoch(time_use/DAY2YEAR)   #relative to t0
    dt = (t2.mjd2000 - t1.mjd2000)*DAY2SEC


    r1, v1 = planetA.eph(t1)
    r2, v2 = planetB.eph(t2)
    
    l = pkp.lambert_problem(r1,r2,dt,mu_star)

    vA = l.get_v1()
    vB = l.get_v2()
    vel =  np.linalg.norm(vA) 
    v_list.append((vel,time_use))
    
v_list = np.array(v_list)
best_time = np.amin(v_list, axis = 0) #contains [v, t]
print 'min v' ,best_time[0]

i = np.argmin(v_list, axis = 0)[0]
optimal_time = best_time[1]
print 'optimal time use:', optimal_time
fig = plt.figure()
ax = fig.gca(projection = '3d')

t1 = epoch(0)            #relative to t0
t2 = epoch(best_time[1]/DAY2YEAR)   #relative to t0
dt = (t2.mjd2000 - t1.mjd2000)*DAY2SEC

r1, v1 = planetA.eph(t1)
r2, v2 = planetB.eph(t2)
l = pkp.lambert_problem(r1,r2,dt,mu_star)
norm = np.linalg.norm
print norm(l.get_v1())
print norm(v1)

plot_planet(planetA, t0 = epoch(0), legend = True, units = AU, ax = ax)
plot_planet(planetB, t0 = epoch(optimal_time/DAY2YEAR), legend = True, units = AU, ax = ax)
plot_lambert(l, legend=True, units =AU, ax=ax)
plt.show()

min v 17949.5899151
optimal time use: 4.1
18563.5168344
17244.8887894
